In [1]:
import os

In [2]:
%pwd

'/Users/mananpatel/Documents/vscode/nlp/Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/mananpatel/Documents/vscode/nlp/Text-Summarizer'

In [6]:
#entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

In [7]:
#configuration manager from src

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


ModuleNotFoundError: No module named 'textSummarizer'